# Nearest neighbors graph

Anton Antonov   
September 2024

-----

## Setup

In [2]:
use Data::Importers;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions::Native;
use Statistics::OutlierIdentifiers;

use NativeCall;

use Math::Nearest;
use Graph;
use JavaScript::D3;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

-------

## Small graph

Here is a set of words:

In [ ]:
my @content = <angel apple ardvark bible car cat cherry chocolate cookie cow devil film horse house movie projector raccoon tiger tree>;

Here we specify LLM-access configuration:

In [ ]:
my $conf = llm-configuration('Gemini');
$conf.Hash.elems

Here we create semantic search index:

In [ ]:
my $vdbObjSmall = create-semantic-search-index(@content, e => $conf, name => 'words')

Here we see the dimensions of the obtained vectors:

In [ ]:
$vdbObjSmall.vectors».elems

Here we find the embedding of a certain word (using the same LLM model as above):

In [ ]:
my $vec = llm-embedding("coffee", e => $conf).head;
$vec.elems

Here we find the closest Nearest Neighbors (NNs) of that word:

In [ ]:
my @nns = $vdbObjSmall.nearest($vec, 3, prop => 'label' ).map(*.Slip)

Here are the corresponding words:

In [ ]:
$vdbObjSmall.items{@nns}

Here we find the corresponding NNs graph with 1 and 2 nns per vertex:

In [ ]:
my ($gr1, $gr2) = [1, 2].map({ 
        # NNs graph edges
        my @edges = nearest-neighbor-graph(
            $vdbObjSmall.vectors.pairs, 
            $_, 
            method => 'Scan', 
            distance-function => &euclidean-distance, 
            format => 'dataset'
        );

        # Replace IDs with names
        @edges .= map({ $_<from> = $vdbObjSmall.items{$_<from>}; $_<to> = $vdbObjSmall.items{$_<to>}; $_ });
        
        # Make the graph
        Graph.new(@edges)
}).flat


Find 1-nns graph's connected components:

In [ ]:
my @comps = $gr1.connected-components.sort(-*.elems)

In [ ]:
#%js

$gr1.edges(:dataset)
==> js-d3-graph-plot(
        :$background,
        highlight => [|@comps.head, |$gr1.subgraph(@comps.head).edges],
        width => 600,
        vertex-label-color => 'Ivory',
        edge-thickness => 2,
        vertex-size => 3,
        vertex-color => 'Blue',
        edge-color => 'SteelBlue',
        force => { charge => {strength => -200, iterations => 4}, collision => {iterations => 1, radius => 10} }
    )

------

## Ingest vector databases

In [ ]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model>;
vector-database-objects(f=>'hash', :flat)
==> to-html(:@field-names)

In [ ]:
#my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<name> ∈ <No833 No747> && $_<llm-service> eq 'gemini' }).map({ create-vector-database(file => $_<file>) })
my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<id> ∈ <266b20ca-d917-4ac0-9b0a-7c420625666c d2effebc-2cef-4b2b-84ca-5dcfa3c1864b> }).map({ create-vector-database(file => $_<file>) })

-----

## Nearest neighbor graph

In [ ]:
$vdbObj = @vdbs.head

Make the nearest neighbor graph for the vectors in the vector database:

In [ ]:
my @edges = nearest-neighbor-graph($vdbObj.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku')

In [ ]:
my sub replaced-ids-with-items(@rules where *.all ~~ Pair:D) {
    @rules = map({ ($vdbObj.items{$_.key} // $_.key) => ($vdbObj.items{$_.value} // $_.value) })
}

Make the graph:

In [ ]:
my $gr = Graph.new(@edges)

Get graph's connected components:

In [ ]:
my @comps = $gr.connected-components.sort(-*.elems);
.say for @comps.head(12)

Example paragraph:

In [ ]:
#% markdown

$vdbObj.items<305.0>

LLM function for naming a set of paragraphs:

In [ ]:
my &fNamer = llm-function({"Summarize the text into a very short sentence that has at most 8 words: \n\n $_"})

Example title finding:

In [ ]:
&fNamer( $vdbObj.items{|@comps[5]}.join("\n") )

Find titles for some of the largest components:

In [ ]:
my @titles = @comps.head(6).map({ &fNamer( $vdbObj.items{|$_}.join("\n") ) });

.say for @titles

Make rules for all components and titles:

In [ ]:
my @res = do for ^@titles.elems -> $i {
    my @vals = @comps[$i].Array X~ ' : ' X~  @titles[$i];
    @comps[$i].Array Z=> @vals
}

my %rules = @res.map(*.Slip);

%rules.elems


Make graph highlight specification:

In [ ]:
my $highlight = (@colors.head(6).Array Z=> @comps.head(6).map({ [ |$_.map({ %rules{$_} // $_ }), |$gr.subgraph($_).edges.map({ ( %rules{$_.key} // $vdbObj.items{$_.key} ) => ( %rules{$_.value} // $vdbObj.items{$_.value} ) }) ]})).Hash;

Semantic graph:

In [ ]:
#%js
my @colors = <#1f77b4 #ff7f0e #2ca02c #d62728 #9467bd #8c564b #e377c2 #7f7f7f #bcbd22 #17becf>;

my @edges2 = @edges.map({ ( %rules{$_.key} // $vdbObj.items{$_.key} ) => ( %rules{$_.value} // $vdbObj.items{$_.value} ) });

@edges2
==> js-d3-graph-plot(
        :$background,
        :%highlight,
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1000,
        height => 900,
        edge-color => 'Gray',
        vertex-color => 'Ivory',
    )

-----

## Combined databases graph

In [ ]:
my $vdbObj2 = vector-database-join(@vdbs)

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[CWv1] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)